In [1]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import HTML, display, JSON

import unstructured_client
from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.staging.base import dict_to_elements, elements_to_json

import json
import os

import chromadb


In [4]:
# Connect to the SQLite database
conn = sqlite3.connect('investopedia_cleaned.db')

# Load all records into a pandas DataFrame
query = "SELECT processed_json FROM investopedia"
df_records = pd.read_sql_query(query, conn )

# Close the connection
conn.close()

In [6]:
# Count the number of records
num_records = len(df_records)
print(f"Number of records: {num_records}")

Number of records: 6288


In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('investopedia_cleaned.db')

# Load all records into a pandas DataFrame
query = "SELECT processed_json FROM investopedia LIMIT 1"
df_record = pd.read_sql_query(query, conn )

# Close the connection
conn.close()

In [3]:
# Load the first record as a JSON object
record = json.loads(df_record['processed_json'][0])
record

[{'type': 'Title',
  'element_id': '7ffe2592fb17a9276e7a6567c2458cd7',
  'text': '0x (ZRX) Protocol: What It Meant and How It Worked',
  'metadata': {'category_depth': 0,
   'languages': ['eng'],
   'filetype': 'text/html'}},
 {'type': 'Title',
  'element_id': '1d6f144c0ca8b86aac2de00ae1523abe',
  'text': 'Close',
  'metadata': {'category_depth': 0,
   'emphasized_text_contents': ['Close'],
   'emphasized_text_tags': ['span'],
   'languages': ['eng'],
   'filetype': 'text/html'}},
 {'type': 'Title',
  'element_id': 'd289e028f60bbbeb5cd3309880545e7c',
  'text': 'What Was the 0x (ZRX) Protocol?',
  'metadata': {'category_depth': 1,
   'emphasized_text_contents': ['What Was the 0x (ZRX) Protocol?'],
   'emphasized_text_tags': ['span'],
   'languages': ['eng'],
   'parent_id': '1d6f144c0ca8b86aac2de00ae1523abe',
   'filetype': 'text/html'}},
 {'type': 'NarrativeText',
  'element_id': '9923862f4a4d5b34ef5900d1f96089fe',
  'text': "The 0x (zero x) protocol enabled the peer-to-peer exchange o

In [10]:
def create_cleaned_json(data):
    exclude_titles = {"Close", "The Bottom Line", "Article Sources", "Compare Accounts"}

    # Remove any Title element with the text "Close"
    data = [el for el in data if not (el['type'] == 'Title' and el['text'] in exclude_titles)]

    # Extract the first title as metadata
    first_title = None
    for el in data:
        if el['type'] == 'Title':
            first_title = el
            break

    # Create a mapping of element_id to title
    title_mapping = {el['element_id']: el for el in data if el['type'] == 'Title'}

    # Create a list to hold the cleaned JSON objects
    cleaned_data = []

    # Iterate over the elements to link NarrativeText with their parent titles
    for el in data:
        if el['type'] == 'NarrativeText':
            parent_id = el['metadata'].get('parent_id')
            if parent_id in title_mapping:
                # Find the corresponding title
                parent_title = title_mapping[parent_id]
                # Check if the title is already in cleaned_data
                title_obj = next((item for item in cleaned_data if item['element_id'] == parent_id), None)
                if title_obj:
                    # Add the NarrativeText to the existing title
                    title_obj['narratives'].append(el)
                else:
                    # Create a new title object and add it to cleaned_data
                    title_obj = parent_title.copy()
                    title_obj['narratives'] = [el]
                    cleaned_data.append(title_obj)

    # Include the first title as metadata
    cleaned_json = {
        'metadata': first_title,
        'data': cleaned_data
    }

    return cleaned_json

# Create the cleaned JSON object
cleaned_json = create_cleaned_json(record)

# Print the cleaned JSON
print(json.dumps(cleaned_json, indent=2))

{
  "metadata": {
    "type": "Title",
    "element_id": "7ffe2592fb17a9276e7a6567c2458cd7",
    "text": "0x (ZRX) Protocol: What It Meant and How It Worked",
    "metadata": {
      "category_depth": 0,
      "languages": [
        "eng"
      ],
      "filetype": "text/html"
    }
  },
  "data": [
    {
      "type": "Title",
      "element_id": "d289e028f60bbbeb5cd3309880545e7c",
      "text": "What Was the 0x (ZRX) Protocol?",
      "metadata": {
        "category_depth": 1,
        "emphasized_text_contents": [
          "What Was the 0x (ZRX) Protocol?"
        ],
        "emphasized_text_tags": [
          "span"
        ],
        "languages": [
          "eng"
        ],
        "parent_id": "1d6f144c0ca8b86aac2de00ae1523abe",
        "filetype": "text/html"
      },
      "narratives": [
        {
          "type": "NarrativeText",
          "element_id": "9923862f4a4d5b34ef5900d1f96089fe",
          "text": "The 0x (zero x) protocol enabled the peer-to-peer exchange of asset

In [13]:
# Function to format the cleaned JSON data

def transform_cleaned_data(data):
    # Extract metadata
    first_title = data["metadata"]
    
    # List to store the transformed data
    transformed_data = []

    # Add the first title to the transformed data
    transformed_data.append(first_title)

    # Process each title and its narratives
    for item in data["data"]:
        # Add the title to the transformed data
        transformed_data.append(item)
        # Add each narrative text associated with the title
        for narrative in item.get("narratives", []):
            transformed_data.append(narrative)
    
    return transformed_data

In [14]:
#Testing format
# Transform the cleaned data
transformed_data = transform_cleaned_data(cleaned_json)

# Convert to JSON
transformed_json = json.dumps(transformed_data, indent=2)

# Print the transformed JSON
print(transformed_json)

[
  {
    "type": "Title",
    "element_id": "7ffe2592fb17a9276e7a6567c2458cd7",
    "text": "0x (ZRX) Protocol: What It Meant and How It Worked",
    "metadata": {
      "category_depth": 0,
      "languages": [
        "eng"
      ],
      "filetype": "text/html"
    }
  },
  {
    "type": "Title",
    "element_id": "d289e028f60bbbeb5cd3309880545e7c",
    "text": "What Was the 0x (ZRX) Protocol?",
    "metadata": {
      "category_depth": 1,
      "emphasized_text_contents": [
        "What Was the 0x (ZRX) Protocol?"
      ],
      "emphasized_text_tags": [
        "span"
      ],
      "languages": [
        "eng"
      ],
      "parent_id": "1d6f144c0ca8b86aac2de00ae1523abe",
      "filetype": "text/html"
    },
    "narratives": [
      {
        "type": "NarrativeText",
        "element_id": "9923862f4a4d5b34ef5900d1f96089fe",
        "text": "The 0x (zero x) protocol enabled the peer-to-peer exchange of assets on the Ethereum blockchain. It was launched in 2017 by ZeroEx Labs, 

In [19]:
# Function to process each record and accumulate the results
def process_and_accumulate_records(df_records, all_cleaned_jsons):
    for index, row in df_records.iterrows():
        record_id = row['id']
        processed_json_str = row['processed_json']
        processed_json = json.loads(processed_json_str)
        
        # Process the record
        cleaned_json = create_cleaned_json(processed_json)
        
        # Add to the list of all cleaned JSON objects
        all_cleaned_jsons.append(cleaned_json)

In [17]:
# Connect to the SQLite database
conn = sqlite3.connect('investopedia_cleaned.db')

# Load all records into a pandas DataFrame
query = "SELECT id, processed_json FROM investopedia"
df_records = pd.read_sql_query(query, conn)

# Ensure the output directory exists
output_dir = "cleaned_json_files"
os.makedirs(output_dir, exist_ok=True)

# List to hold all cleaned JSON objects
all_cleaned_jsons = []

# Process all records
process_and_accumulate_records(df_records, all_cleaned_jsons)

# Convert the list of all cleaned JSON objects to a JSON string
all_cleaned_json_str = json.dumps(all_cleaned_jsons, indent=2)

# Define the output file path
output_file_path = "all_cleaned_records.txt"

# Save all cleaned JSON to a single text file
with open(output_file_path, 'w') as output_file:
    output_file.write(all_cleaned_json_str)

# Close the connection
conn.close()

print(f"All cleaned JSON records have been saved to: {output_file_path}")


All cleaned JSON records have been saved to: all_cleaned_records.txt


In [23]:
# Search for titles containing 'protocol'
titles_with_protocol = []

for record in all_cleaned_jsons:
    # Check the metadata title first
    if record.get('metadata') and record['metadata'].get('text') and 'stock' in record['metadata']['text'].lower():
        titles_with_protocol.append(record['metadata'])
    
    # Check the titles within the data
    for item in record.get('data', []):
        if item.get('type') == 'Title' and item.get('text') and 'stock' in item['text'].lower():
            titles_with_protocol.append(item)

# Print the found titles
print(json.dumps(titles_with_protocol, indent=2))


[
  {
    "type": "Title",
    "element_id": "aa57dfff1cba93ad5ccf37ad6dd8751a",
    "text": "130-30 Strategy and Shorting Stocks",
    "metadata": {
      "category_depth": 1,
      "emphasized_text_contents": [
        "130-30 Strategy and Shorting Stocks"
      ],
      "emphasized_text_tags": [
        "span"
      ],
      "languages": [
        "eng"
      ],
      "parent_id": "1d6f144c0ca8b86aac2de00ae1523abe",
      "filetype": "text/html"
    },
    "narratives": [
      {
        "type": "NarrativeText",
        "element_id": "4540fdc00ec36ba72f547e107e921f30",
        "text": "The 130-30 strategy incorporates short sales as a significant part of its activity. Shorting a stock entails borrowing securities from another party, most often a broker, and agreeing to pay an interest rate as a fee. A negative position is subsequently recorded in the investor\u2019s account. The investor then sells the newly acquired securities on the open market at the current price and receives th

In [ ]:
# Step 1: Read the JSON data from the file
with open('all_cleaned_records.txt', 'r') as file:
    all_cleaned_jsons = json.load(file)

# Step 2: Initialize the Chroma client
client = chromadb.PersistentClient(path="chroma_tmp", settings=chromadb.Settings(allow_reset=True))
client.reset()

# Step 3: Create a collection
collection = client.create_collection(
    name="investopedia_articles_2",
    metadata={"hnsw:space": "cosine"}
)

# Step 4: Prepare elements for the collection
elements_list = []

# Iterate through the cleaned JSON objects and prepare the elements for the collection
for record in all_cleaned_jsons:
    # Add the metadata title first
    metadata = record.get('metadata')
    if metadata and metadata.get('text'):
        elements_list.append({
            "text": metadata['text'],
            "element_id": metadata['element_id'],
            "metadata": {"type": metadata['type'], "is_title": True, "element_id": metadata['element_id']}
        })

    # Add the titles and narratives within the data
    for item in record.get('data', []):
        if item.get('type') == 'Title' and item.get('text'):
            elements_list.append({
                "text": item['text'],
                "element_id": item['element_id'],
                "metadata": {"type": item['type'], "is_title": True, "element_id": item['element_id']}
            })
        if 'narratives' in item:
            for narrative in item['narratives']:
                elements_list.append({
                    "text": narrative['text'],
                    "element_id": narrative['element_id'],
                    "metadata": {"type": narrative['type'], "is_title": False, "parent_title_id": item['element_id']}
                })

# Step 5: Add elements to the collection
for element in elements_list:
    collection.add(
        documents=[element["text"]],
        ids=[element["element_id"]],
        metadatas=[element["metadata"]]
    )

# Step 6: Peek at the collection to ensure elements are added
results = collection.peek()
print(results["documents"])
